### 정규화 - dropout

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import torch

USE_CUDA = torch.cuda.is_available()
#                gpu  사용가능여부(True / False)
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 5
BATCH_SIZE = 64

In [4]:
from torchvision import transforms, datasets

#mnist 데이터셋 다운로드, transforms.Compose 텐서로 바꾸면서 0~1 정규화
train_loader = torch.utils.data.DataLoader(

    datasets.MNIST('c:/workspace3/data',
#                       저장경로
                   train=True,
                   download=True,
                   transform=transforms.Compose([transforms.RandomHorizontalFlip(),
#                                                             랜덤 가로방향 전환    
                                                 transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))])),
#                                                                        평균        표준편차
                   batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('c:/workspace3/data',
                   train=False,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))])),
                   batch_size=BATCH_SIZE, shuffle=True)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
#                            in , out
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)     # 1차원 픽셀 784
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = self.fc3(x)
        return x

In [6]:
import torch.optim as optim

#과적합과 정규화
model        = Net(dropout_p=0.2).to(DEVICE)
#                    드롭아웃 비율     cpu / gpu
optimizer    = optim.SGD(model.parameters(), lr=0.01)
#                  확률적 경사하강법            학습률

In [7]:
def train(model, train_loader, optimizer):

    model.train()
#          학습모드 : 순전파 + 역전파
#          추론모드 : 순전파

    for _, (data, target) in enumerate(train_loader):
#                                인덱스, 데이터
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
#                   경사 초기화
        output = model(data)
#                모델   입력
        loss = F.cross_entropy(output, target)
        loss.backward() # 역전파 계산
        optimizer.step() # 수정

In [8]:
def evaluate(model, test_loader):
    model.eval()
#          추론모드 : 순전파

    test_loss = 0
    correct = 0

    with torch.no_grad():
#              미분 X
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            #크로스 엔트로피 계산
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            # softmax 연산(output에서 최대값 선택)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)    
    print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

[1] Test Loss: 0.5592, Accuracy: 82.21%
[2] Test Loss: 0.4303, Accuracy: 86.45%
[3] Test Loss: 0.3554, Accuracy: 89.13%
[4] Test Loss: 0.2971, Accuracy: 90.86%
